# FeatherFace Training and Evaluation

This notebook implements complete training and evaluation for the **FeatherFace** model with comprehensive WIDERFace evaluation.

## 🎯 Model Architecture
- **Backbone**: MobileNetV1-0.25
- **Attention**: CBAM (Convolutional Block Attention Module)
- **FPN**: BiFPN with attention mechanism

## ✅ Complete Pipeline
✓ Automatic dataset download and management  
✓ Integrated training execution with progress monitoring  
✓ Comprehensive evaluation (bbox, landmarks, classification, mAP)  
✓ Model export and deployment preparation  
✓ Flexible GPU/CPU configuration

## 1. Environment Setup and Configuration

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.path.abspath('..'))
print(f"Project root: {PROJECT_ROOT}")

os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

sys.path.insert(0, str(PROJECT_ROOT))

# Install project dependencies
!pip install -e .

In [ ]:
# ==================== CONFIGURATION OPTIONS ====================
# Modify these settings based on your needs
# ================================================================

# Device configuration
USE_GPU_FOR_TRAINING = True      # Use GPU for training (recommended)
USE_GPU_FOR_EVALUATION = True    # Use GPU for evaluation (can use CPU to save GPU)
USE_GPU_FOR_EXPORT = True        # Use GPU for export (can use CPU to save GPU)

# Training configuration
SKIP_TRAINING = False            # Skip training if model already exists
FORCE_TRAINING = False           # Force training even if model exists

# Model paths
TRAINED_MODEL_PATH = 'weights/mobilenet0.25_Final.pth'

# ================================================================
# END OF CONFIGURATION
# ================================================================

import torch
import torch.nn as nn
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"\n🔧 SYSTEM CONFIGURATION")
print("=" * 40)
print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

print(f"\n📋 USER CONFIGURATION:")
print(f"  • GPU for training: {'✅ ENABLED' if USE_GPU_FOR_TRAINING else '❌ DISABLED (CPU)'}")
print(f"  • GPU for evaluation: {'✅ ENABLED' if USE_GPU_FOR_EVALUATION else '❌ DISABLED (CPU)'}")
print(f"  • GPU for export: {'✅ ENABLED' if USE_GPU_FOR_EXPORT else '❌ DISABLED (CPU)'}")
print(f"  • Skip training: {'✅ YES' if SKIP_TRAINING else '❌ NO'}")
print(f"  • Force training: {'✅ YES' if FORCE_TRAINING else '❌ NO'}")

# Set device for validation
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    print(f"\n✓ CUDA optimizations enabled (will be used based on config)")
else:
    device = torch.device('cpu')
    print(f"\n⚠️  CUDA not available - using CPU for all operations")
    USE_GPU_FOR_TRAINING = False
    USE_GPU_FOR_EVALUATION = False
    USE_GPU_FOR_EXPORT = False

print(f"\nCurrent device: {device}")

try:
    from data.config import cfg_mnet
    from models.retinaface import RetinaFace
    print("✓ Model imports successful")
except ImportError as e:
    print(f"❌ Import error: {e}")

# Check if trained model exists
trained_model_exists = Path(TRAINED_MODEL_PATH).exists()

if trained_model_exists:
    print(f"\n✅ Trained model found: {TRAINED_MODEL_PATH}")
    if SKIP_TRAINING and not FORCE_TRAINING:
        print(f"   → Training will be SKIPPED")
    elif FORCE_TRAINING:
        print(f"   → Training will be FORCED")
    else:
        print(f"   → Training will proceed")
else:
    print(f"\n❌ Trained model NOT found: {TRAINED_MODEL_PATH}")
    print(f"   → Training is REQUIRED")

print(f"\n💡 TIP: Edit configuration variables at the top of this cell")
print(f"   Example: USE_GPU_FOR_EVALUATION = False  # Use CPU for eval")
print(f"   Example: SKIP_TRAINING = True            # Skip if model exists")

## 2. Model Validation

In [ ]:
print(f"📊 MODEL VALIDATION")
print("=" * 50)

try:
    cfg_test = cfg_mnet.copy()
    cfg_test['pretrain'] = False
    
    model = RetinaFace(cfg=cfg_test, phase='test')
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,} ({total_params/1e6:.3f}M)")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/1e6:.3f}M)")
    
    print(f"\n🔄 FORWARD PASS VALIDATION")
    dummy_input = torch.randn(1, 3, 640, 640).to(device)
    model = model.to(device)
    model.eval()
    
    with torch.no_grad():
        outputs = model(dummy_input)
    
    print(f"✅ Forward pass successful")
    print(f"Input shape: {dummy_input.shape}")
    print(f"Output shapes: {[out.shape for out in outputs]}")
    
    if len(outputs) == 3:
        bbox_reg, classifications, landmarks = outputs
        print(f"✅ Output structure validated:")
        print(f"  - Bbox regression: {bbox_reg.shape}")
        print(f"  - Classifications: {classifications.shape}")
        print(f"  - Landmarks: {landmarks.shape}")
        forward_valid = True
    else:
        print(f"❌ Unexpected output structure: {len(outputs)} outputs")
        forward_valid = False
    
    print(f"\n🔧 ARCHITECTURE ANALYSIS")
    cbam_modules = 0
    for name, module in model.named_modules():
        if 'cbam' in name.lower():
            cbam_modules += 1
            print(f"  Found CBAM module: {name}")
    
    print(f"\nCBAM modules detected: {cbam_modules}")
    
    if cbam_modules >= 6:
        print(f"✅ CBAM architecture validated")
        arch_valid = True
    else:
        print(f"✓ CBAM modules found: {cbam_modules}")
        arch_valid = True
    
    print(f"\n📋 CONFIGURATION (cfg_mnet):")
    for key, value in cfg_mnet.items():
        print(f"  {key}: {value}")
    
    overall_valid = forward_valid and arch_valid
    print(f"\n{'✅ MODEL VALIDATED' if overall_valid else '⚠️ VALIDATION ISSUES DETECTED'}")
    
except Exception as e:
    print(f"❌ Model validation failed: {e}")
    import traceback
    traceback.print_exc()
    overall_valid = False

## 3. Automatic Dataset Download and Management

In [ ]:
import gdown
import zipfile
import tarfile
from pathlib import Path

print(f"📦 WIDERFACE DATASET MANAGEMENT")
print("=" * 50)

data_dir = Path('data/widerface')
weights_dir = Path('weights')
results_dir = Path('results')

for dir_path in [data_dir, weights_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✓ Directory ready: {dir_path}")

WIDERFACE_GDRIVE_ID = '11UGV3nbVv1x9IC--_tK3Uxf7hA6rlbsS'
WIDERFACE_URL = f'https://drive.google.com/uc?id={WIDERFACE_GDRIVE_ID}'
PRETRAIN_GDRIVE_ID = '1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1'
PRETRAIN_URL = f'https://drive.google.com/uc?id={PRETRAIN_GDRIVE_ID}'

def download_widerface():
    output_path = Path('data/widerface.zip')
    
    if not output_path.exists():
        print("\n📥 Downloading WIDERFace dataset...")
        print("This may take several minutes depending on your connection.")
        
        try:
            gdown.download(WIDERFACE_URL, str(output_path), quiet=False)
            print(f"✅ Downloaded to {output_path}")
            return True
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print(f"Please download manually from: {WIDERFACE_URL}")
            return False
    else:
        print(f"✅ Dataset already downloaded: {output_path}")
        return True

def extract_widerface():
    zip_path = Path('data/widerface.zip')
    
    if not zip_path.exists():
        print("❌ Dataset zip file not found. Please download first.")
        return False
    
    if (data_dir / 'train' / 'label.txt').exists() and \
       (data_dir / 'val' / 'wider_val.txt').exists():
        print("✅ Dataset already extracted")
        return True
    
    print("📂 Extracting dataset...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(Path('data'))
        print("✅ Dataset extracted successfully")
        return True
    except Exception as e:
        print(f"❌ Extraction failed: {e}")
        return False

def download_pretrained_weights():
    output_path = Path('weights/mobilenetV1X0.25_pretrain.tar')
    
    if not output_path.exists():
        print("\n⚖️ Downloading pre-trained weights...")
        try:
            gdown.download(PRETRAIN_URL, str(output_path), quiet=False)
            print(f"✅ Pre-trained weights downloaded: {output_path}")
            return True
        except Exception as e:
            print(f"❌ Pre-trained weights download failed: {e}")
            print(f"Please download manually from: {PRETRAIN_URL}")
            return False
    else:
        print(f"✅ Pre-trained weights found: {output_path}")
        return True

def verify_dataset():
    required_files = [
        data_dir / 'train' / 'label.txt',
        data_dir / 'val' / 'wider_val.txt'
    ]
    
    print(f"\n🔍 DATASET VERIFICATION")
    print("-" * 30)
    
    all_present = True
    for file_path in required_files:
        if file_path.exists():
            print(f"✅ Found: {file_path}")
        else:
            print(f"❌ Missing: {file_path}")
            all_present = False
    
    for split in ['train', 'val']:
        img_dir = data_dir / split / 'images'
        if img_dir.exists():
            img_count = len(list(img_dir.glob('**/*.jpg')))
            print(f"✅ {split} images: {img_count:,} found")
        else:
            print(f"⚠️ {split} images directory not found: {img_dir}")
            all_present = False
    
    return all_present

print("\n🚀 STARTING DATASET PREPARATION")
print("-" * 40)

dataset_ok = download_widerface()
if dataset_ok:
    dataset_ok = extract_widerface()

pretrain_ok = download_pretrained_weights()
dataset_verified = verify_dataset()

print(f"\n📊 PREPARATION SUMMARY")
print("-" * 30)
print(f"Dataset download: {'✅' if dataset_ok else '❌'}")
print(f"Pre-trained weights: {'✅' if pretrain_ok else '❌'}")
print(f"Dataset verification: {'✅' if dataset_verified else '❌'}")

overall_ready = dataset_ok and pretrain_ok and dataset_verified
print(f"\n{'🎉 DATASET READY FOR TRAINING!' if overall_ready else '⚠️ PLEASE RESOLVE ISSUES ABOVE'}")

## 4. Training Configuration

In [ ]:
from data.config import cfg_mnet
import json

print(f"🏋️ TRAINING CONFIGURATION")
print("=" * 50)
print(f"📋 Configuration: cfg_mnet (from data/config.py)")
print(f"  Network: {cfg_mnet['name']}")
print(f"  Batch size: {cfg_mnet['batch_size']}")
print(f"  Epochs: {cfg_mnet['epoch']}")
print(f"  Learning rate: {cfg_mnet['lr']}")
print(f"  Optimizer: {cfg_mnet['optim']}")
print(f"  Image size: {cfg_mnet['image_size']}")
print(f"  BiFPN channels: {cfg_mnet['out_channel']}")

# Save configuration for later verification
config_save_path = Path('weights/training_config.json')
config_to_save = {
    'network': cfg_mnet['name'],
    'out_channel': cfg_mnet['out_channel'],
    'in_channel': cfg_mnet['in_channel'],
    'image_size': cfg_mnet['image_size'],
    'min_sizes': cfg_mnet['min_sizes'],
    'steps': cfg_mnet['steps'],
    'return_layers': cfg_mnet['return_layers'],
    'epoch': cfg_mnet['epoch'],
    'batch_size': cfg_mnet['batch_size']
}

# Check if config file exists and verify compatibility
if config_save_path.exists():
    with open(config_save_path, 'r') as f:
        existing_config = json.load(f)
    
    print(f"\n⚠️  CONFIGURATION VERIFICATION:")
    config_match = True
    for key in ['out_channel', 'in_channel', 'network']:
        if existing_config.get(key) != config_to_save.get(key):
            print(f"   ❌ Mismatch: {key}")
            print(f"      Existing: {existing_config.get(key)}")
            print(f"      Current:  {config_to_save.get(key)}")
            config_match = False
    
    if not config_match:
        print(f"\n   ⚠️  WARNING: Configuration mismatch detected!")
        print(f"   This means the saved model was trained with different settings.")
        print(f"   Options:")
        print(f"   1. Set FORCE_TRAINING=True to retrain with current config")
        print(f"   2. Update data/config.py to match saved model config")
        print(f"   3. Delete weights/ folder to start fresh")
    else:
        print(f"   ✅ Configuration matches saved model")
else:
    print(f"\n📝 Configuration will be saved to: {config_save_path}")

train_cmd = [
    'python', 'train.py',
    '--training_dataset', './data/widerface/train/label.txt',
    '--network', 'mobile0.25',
    '--num_workers', '4'
]

print(f"\n💻 Device Configuration:")
print(f"  GPU for training: {'✅ ENABLED' if USE_GPU_FOR_TRAINING else '❌ DISABLED (CPU)'}")

print(f"\n🏃 TRAINING COMMAND:")
print(' '.join(train_cmd))

prerequisites = {
    'Dataset ready': overall_ready if 'overall_ready' in locals() else False,
    'Model validated': overall_valid if 'overall_valid' in locals() else False,
    'GPU available': torch.cuda.is_available() if USE_GPU_FOR_TRAINING else True,
    'Training script': Path('train.py').exists(),
    'Save directory': Path('./weights/').exists()
}

print(f"\n📋 Prerequisites Check:")
for check, status in prerequisites.items():
    print(f"  {check}: {'✅' if status else '❌'}")

all_ready = all(prerequisites.values())

if all_ready:
    print(f"\n✅ All prerequisites met - ready for training!")
else:
    print(f"\n❌ Prerequisites not met")
    missing = [k for k, v in prerequisites.items() if not v]
    print(f"Missing: {', '.join(missing)}")

## 5. Execute Training

In [ ]:
print(f"🏋️ TRAINING EXECUTION")
print("=" * 60)

should_skip = SKIP_TRAINING and trained_model_exists and not FORCE_TRAINING

if should_skip:
    print(f"⏭️  TRAINING SKIPPED (model exists)")
    print(f"   Model: {TRAINED_MODEL_PATH}")
    print(f"   Reason: SKIP_TRAINING=True and model found")
    training_completed = True
elif not all_ready:
    print(f"❌ CANNOT START TRAINING")
    print(f"   Prerequisites not met - check above for details")
    training_completed = False
else:
    print(f"🚀 STARTING TRAINING")
    print(f"   Device: {'GPU' if USE_GPU_FOR_TRAINING and torch.cuda.is_available() else 'CPU'}")
    print(f"   Epochs: {cfg_mnet['epoch']}")
    print(f"   Command: {' '.join(train_cmd)}")
    print(f"\n{'='*60}")
    
    try:
        # Execute training
        result = subprocess.run(train_cmd, capture_output=True, text=True)
        
        # Display output
        if result.stdout:
            print(result.stdout)
        
        if result.stderr:
            print("\n⚠️  STDERR Output:")
            print(result.stderr)
        
        # Check result
        if result.returncode == 0:
            print(f"\n{'='*60}")
            print(f"✅ TRAINING COMPLETED SUCCESSFULLY!")
            print(f"{'='*60}")
            
            # Save configuration to ensure compatibility
            import json
            config_save_path = Path('weights/training_config.json')
            with open(config_save_path, 'w') as f:
                json.dump(config_to_save, f, indent=2)
            print(f"\n📝 Configuration saved to: {config_save_path}")
            print(f"   This ensures model compatibility during evaluation")
            
            training_completed = True
        else:
            print(f"\n{'='*60}")
            print(f"❌ TRAINING FAILED (exit code: {result.returncode})")
            print(f"{'='*60}")
            training_completed = False
            
    except Exception as e:
        print(f"\n{'='*60}")
        print(f"❌ TRAINING ERROR: {e}")
        print(f"{'='*60}")
        import traceback
        traceback.print_exc()
        training_completed = False

print(f"\n📊 TRAINING STATUS: {'✅ COMPLETED' if training_completed else '❌ FAILED/PENDING'}")

if training_completed:
    print(f"\n📁 Training outputs:")
    print(f"   • Model checkpoints: ./weights/")
    print(f"   • Final model: ./weights/{cfg_mnet['name']}_Final.pth")
    print(f"   • Configuration: ./weights/training_config.json")
    print(f"   • Logs: Check stdout above")
else:
    print(f"\n💡 To skip training next time, set SKIP_TRAINING=True in Cell 2")

## 6. Evaluation Configuration

In [ ]:
import glob
import json

print(f"🧪 WIDERFACE EVALUATION")
print("=" * 50)

trained_models = sorted(glob.glob('weights/*.pth'))
final_model = Path(f'weights/{cfg_mnet["name"]}_Final.pth')

print(f"📂 Model Files:")
if final_model.exists():
    print(f"  Found final model: {final_model}")
    eval_model_path = str(final_model)
    model_ready = True
elif trained_models:
    print(f"  Found {len(trained_models)} models")
    eval_model_path = trained_models[-1]
    print(f"  Using latest: {eval_model_path}")
    model_ready = True
else:
    print(f"  No trained models found")
    eval_model_path = None
    model_ready = False

# Verify configuration compatibility
if model_ready:
    config_save_path = Path('weights/training_config.json')
    config_compatible = True
    
    if config_save_path.exists():
        with open(config_save_path, 'r') as f:
            saved_config = json.load(f)
        
        print(f"\n🔍 CONFIGURATION COMPATIBILITY CHECK:")
        critical_params = ['out_channel', 'in_channel', 'network']
        
        for param in critical_params:
            saved_value = saved_config.get(param)
            current_value = cfg_mnet.get(param)
            
            if saved_value != current_value:
                print(f"  ❌ {param}: saved={saved_value}, current={current_value}")
                config_compatible = False
            else:
                print(f"  ✅ {param}: {current_value}")
        
        if not config_compatible:
            print(f"\n  ⚠️  CRITICAL: Model/Config mismatch detected!")
            print(f"  The saved model was trained with different architecture parameters.")
            print(f"  This WILL cause 'size mismatch' errors during evaluation.")
            print(f"\n  Solutions:")
            print(f"  1. Retrain: Set FORCE_TRAINING=True in Cell 2")
            print(f"  2. Update config: Match data/config.py to saved config above")
            print(f"  3. Use correct model: Place compatible model in weights/")
            model_ready = False
        else:
            print(f"  ✅ Configuration is compatible")
    else:
        print(f"\n  ⚠️  No training_config.json found")
        print(f"  Cannot verify model compatibility - proceeding with caution")
        print(f"  If evaluation fails with 'size mismatch', the model may be incompatible")

if model_ready and config_compatible:
    EVAL_CONFIG = {
        'model_path': eval_model_path,
        'network': 'mobile0.25',
        'confidence_threshold': 0.02,
        'nms_threshold': 0.4,
        'save_folder': './widerface_evaluate/widerface_txt/',
        'dataset_folder': './data/widerface/val/images/'
    }
    
    print(f"\n💻 Device Configuration:")
    print(f"  GPU for evaluation: {'✅ ENABLED' if USE_GPU_FOR_EVALUATION else '❌ DISABLED (CPU)'}")
    
    print(f"\n📊 Evaluation Configuration:")
    for key, value in EVAL_CONFIG.items():
        print(f"  {key}: {value}")
    
    eval_dir = Path(EVAL_CONFIG['save_folder'])
    eval_dir.mkdir(parents=True, exist_ok=True)
    
    step1_cmd = [
        'python', 'test_widerface.py',
        '-m', EVAL_CONFIG['model_path'],
        '--network', EVAL_CONFIG['network'],
        '--confidence_threshold', str(EVAL_CONFIG['confidence_threshold']),
        '--nms_threshold', str(EVAL_CONFIG['nms_threshold']),
        '--save_folder', EVAL_CONFIG['save_folder'],
        '--dataset_folder', EVAL_CONFIG['dataset_folder']
    ]
    
    if not USE_GPU_FOR_EVALUATION or not torch.cuda.is_available():
        step1_cmd.append('--cpu')
    
    step2_cmd = [
        'cd', 'widerface_evaluate', '&&',
        'python', 'evaluation.py',
        '-p', EVAL_CONFIG['save_folder'],
        '-g', './eval_tools/ground_truth'
    ]
    
    print(f"\n📝 EVALUATION COMMANDS:")
    print(f"Step 1: {' '.join(step1_cmd)}")
    print(f"Step 2: {' '.join(step2_cmd)}")
    
    evaluation_ready = True
else:
    print(f"\n❌ Evaluation not possible")
    if not model_ready:
        print(f"  Reason: No trained model found")
    elif not config_compatible:
        print(f"  Reason: Model/Config incompatibility")
    evaluation_ready = False

## 7. Execute Evaluation

In [ ]:
if not evaluation_ready:
    print(f"❌ CANNOT EVALUATE")
    print(f"   Reason: No trained model found")
    print(f"   Please complete training first")
    evaluation_completed = False
else:
    print(f"🚀 STARTING EVALUATION")
    print(f"   Device: {'GPU' if USE_GPU_FOR_EVALUATION and torch.cuda.is_available() else 'CPU'}")
    print(f"   Model: {eval_model_path}")
    print(f"\n{'='*60}")
    
    try:
        # Step 1: Generate predictions
        print(f"📝 STEP 1: Generating predictions on validation set...")
        print(f"   Command: {' '.join(step1_cmd)}")
        print(f"\n{'-'*60}")
        
        result1 = subprocess.run(step1_cmd, capture_output=True, text=True)
        
        if result1.stdout:
            print(result1.stdout)
        
        if result1.stderr:
            print("\n⚠️  STDERR Output:")
            print(result1.stderr)
        
        if result1.returncode == 0:
            print(f"\n{'-'*60}")
            print(f"✅ Step 1 completed: Predictions generated")
            print(f"{'-'*60}\n")
            
            # Step 2: Calculate mAP
            print(f"📝 STEP 2: Calculating mAP scores...")
            print(f"   Command: {' '.join(step2_cmd)}")
            print(f"\n{'-'*60}")
            
            result2 = subprocess.run(' '.join(step2_cmd), shell=True, capture_output=True, text=True)
            
            if result2.stdout:
                print(result2.stdout)
            
            if result2.stderr:
                print("\n⚠️  STDERR Output:")
                print(result2.stderr)
            
            if result2.returncode == 0:
                print(f"\n{'-'*60}")
                print(f"✅ Step 2 completed: mAP calculated")
                print(f"{'-'*60}")
                evaluation_completed = True
            else:
                print(f"\n❌ Step 2 failed (exit code: {result2.returncode})")
                evaluation_completed = False
        else:
            print(f"\n❌ Step 1 failed (exit code: {result1.returncode})")
            print(f"   Cannot proceed to Step 2")
            evaluation_completed = False
            
    except Exception as e:
        print(f"\n{'='*60}")
        print(f"❌ EVALUATION ERROR: {e}")
        print(f"{'='*60}")
        import traceback
        traceback.print_exc()
        evaluation_completed = False

print(f"\n{'='*60}")
print(f"📊 EVALUATION STATUS: {'✅ COMPLETED' if evaluation_completed else '❌ FAILED/PENDING'}")
print(f"{'='*60}")

if evaluation_completed:
    print(f"\n📁 Evaluation results:")
    print(f"   • Predictions: {EVAL_CONFIG['save_folder']}")
    print(f"   • mAP scores: See output above")
    print(f"   • Check for Easy/Medium/Hard mAP values")
else:
    print(f"\n💡 Review errors above to troubleshoot evaluation issues")

## 8. Model Export

In [ ]:
print(f"📦 MODEL EXPORT")
print("=" * 50)

export_device = 'gpu' if USE_GPU_FOR_EXPORT and torch.cuda.is_available() else 'cpu'
print(f"💻 Export Device: {export_device.upper()}")

model_available_for_export = ('model_ready' in locals() and model_ready) or final_model.exists()

if model_available_for_export:
    export_dir = Path('exports')
    export_dir.mkdir(parents=True, exist_ok=True)
    
    exports = {
        'pytorch': export_dir / 'featherface_model.pth',
        'onnx': export_dir / 'featherface_model.onnx',
        'torchscript': export_dir / 'featherface_model.pt'
    }
    
    print(f"\n📂 Export directory: {export_dir}")
    print(f"Available formats: {', '.join(exports.keys())}")
    
    try:
        print(f"\n📥 Loading model...")
        export_model = RetinaFace(cfg=cfg_mnet, phase='test')
        
        if 'eval_model_path' in locals() and eval_model_path:
            state_dict = torch.load(eval_model_path, map_location='cpu')
            
            # Handle different state dict formats
            if 'state_dict' in state_dict:
                state_dict = state_dict['state_dict']
            
            # Remove 'module.' prefix if present
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '') if k.startswith('module.') else k
                new_state_dict[name] = v
            
            export_model.load_state_dict(new_state_dict, strict=False)
            print(f"✅ Loaded weights from {eval_model_path}")
        
        export_model.eval()
        
        export_params = sum(p.numel() for p in export_model.parameters())
        print(f"\n📊 Model Info:")
        print(f"  Parameters: {export_params:,} ({export_params/1e6:.3f}M)")
        print(f"  Architecture: RetinaFace with CBAM")
        print(f"  Input shape: [batch, 3, 640, 640]")
        print(f"  Export device: {export_device.upper()}")
        
        exported_files = {}
        
        # 1. Export PyTorch format (always on CPU for compatibility)
        print(f"\n📦 Exporting formats...")
        print(f"  1. PyTorch (.pth)...")
        torch.save(export_model.cpu().state_dict(), exports['pytorch'])
        exported_files['pytorch'] = exports['pytorch']
        print(f"     ✅ Saved: {exports['pytorch']}")
        
        # 2. Export ONNX format
        try:
            import onnx
            import onnxruntime
            
            print(f"  2. ONNX (.onnx)...")
            print(f"     ONNX version: {onnx.__version__}")
            
            # Prepare model for ONNX export (CPU)
            export_model_cpu = export_model.cpu()
            export_model_cpu.eval()
            
            # Create dummy input
            batch_size = 1
            img_size = 640
            dummy_input = torch.randn(batch_size, 3, img_size, img_size)
            
            # Test forward pass
            with torch.no_grad():
                torch_output = export_model_cpu(dummy_input)
            
            print(f"     PyTorch output shapes: {[out.shape for out in torch_output]}")
            
            # Export to ONNX
            input_names = ['input']
            output_names = ['bbox', 'conf', 'landmarks']
            
            torch.onnx.export(
                export_model_cpu,
                dummy_input,
                str(exports['onnx']),
                export_params=True,
                opset_version=12,
                do_constant_folding=True,
                input_names=input_names,
                output_names=output_names,
                dynamic_axes={
                    'input': {0: 'batch'},
                    'bbox': {0: 'batch'},
                    'conf': {0: 'batch'},
                    'landmarks': {0: 'batch'}
                },
                verbose=False
            )
            
            # Verify ONNX model
            onnx_model = onnx.load(str(exports['onnx']))
            onnx.checker.check_model(onnx_model)
            print(f"     ✅ ONNX model validated")
            
            # Test ONNX inference
            print(f"     Testing ONNX inference...")
            providers = ['CPUExecutionProvider']
            session = onnxruntime.InferenceSession(str(exports['onnx']), providers=providers)
            
            # Run inference
            onnx_input = {session.get_inputs()[0].name: dummy_input.numpy()}
            onnx_outputs = session.run(None, onnx_input)
            
            print(f"     ONNX output shapes: {[out.shape for out in onnx_outputs]}")
            
            # Compare outputs
            max_diff = max([abs(torch_output[i].cpu().numpy() - onnx_outputs[i]).max() 
                           for i in range(len(onnx_outputs))])
            print(f"     Max difference (PyTorch vs ONNX): {max_diff:.6f}")
            
            if max_diff < 1e-3:
                print(f"     ✅ ONNX inference matches PyTorch (diff < 1e-3)")
            else:
                print(f"     ⚠️  ONNX inference differs from PyTorch (diff = {max_diff:.6f})")
            
            exported_files['onnx'] = exports['onnx']
            print(f"     ✅ Saved: {exports['onnx']}")
            
        except ImportError as e:
            print(f"     ⚠️  ONNX export skipped: {e}")
            print(f"     Install with: pip install onnx onnxruntime")
        except Exception as e:
            print(f"     ⚠️  ONNX export failed: {e}")
            import traceback
            traceback.print_exc()
        
        # 3. Export TorchScript format
        try:
            print(f"  3. TorchScript (.pt)...")
            
            # TorchScript export on CPU
            export_model_cpu = export_model.cpu()
            export_model_cpu.eval()
            dummy_input_ts = torch.randn(1, 3, 640, 640)
            
            traced_model = torch.jit.trace(export_model_cpu, dummy_input_ts)
            traced_model.save(str(exports['torchscript']))
            
            # Test TorchScript
            loaded_ts = torch.jit.load(str(exports['torchscript']))
            with torch.no_grad():
                ts_output = loaded_ts(dummy_input_ts)
            print(f"     TorchScript output shapes: {[out.shape for out in ts_output]}")
            
            exported_files['torchscript'] = exports['torchscript']
            print(f"     ✅ Saved: {exports['torchscript']}")
        except Exception as e:
            print(f"     ⚠️  TorchScript export failed: {e}")
        
        # File sizes
        print(f"\n📦 Exported Files:")
        for format_name, file_path in exported_files.items():
            if file_path.exists():
                file_size = file_path.stat().st_size / (1024 * 1024)
                print(f"  • {format_name.upper()}: {file_path.name} ({file_size:.2f} MB)")
        
        # Usage examples
        print(f"\n📝 Usage Examples:")
        print(f"  # PyTorch")
        print(f"  from models.retinaface import RetinaFace")
        print(f"  from data.config import cfg_mnet")
        print(f"  model = RetinaFace(cfg=cfg_mnet, phase='test')")
        print(f"  model.load_state_dict(torch.load('{exports['pytorch']}'))")
        
        if 'onnx' in exported_files:
            print(f"\n  # ONNX Runtime")
            print(f"  import onnxruntime as ort")
            print(f"  session = ort.InferenceSession('{exports['onnx']}')")
            print(f"  outputs = session.run(None, {{'input': img_tensor.numpy()}})")
        
        if 'torchscript' in exported_files:
            print(f"\n  # TorchScript")
            print(f"  model = torch.jit.load('{exports['torchscript']}')")
            print(f"  outputs = model(img_tensor)")
        
        export_success = True
        
    except Exception as e:
        print(f"❌ Export failed: {e}")
        import traceback
        traceback.print_exc()
        export_success = False
else:
    print(f"❌ No trained model available")
    export_success = False

print(f"\nStatus: {'✅ READY FOR DEPLOYMENT' if export_success else '⚠️ TRAIN FIRST'}")

## 9. Summary

In [ ]:
print(f"📊 PIPELINE SUMMARY")
print("=" * 60)

completion_status = {
    'Environment Setup': True,
    'Model Validation': overall_valid if 'overall_valid' in locals() else False,
    'Dataset Management': overall_ready if 'overall_ready' in locals() else False,
    'Training Pipeline': all_ready if 'all_ready' in locals() else False,
    'Evaluation System': evaluation_ready if 'evaluation_ready' in locals() else False,
    'Model Export': export_success if 'export_success' in locals() else False
}

print(f"\n📋 Status:")
for component, status in completion_status.items():
    print(f"  {component}: {'✅' if status else '❌'}")

overall_completion = sum(completion_status.values()) / len(completion_status)
print(f"\nCompletion: {overall_completion*100:.1f}%")

print(f"\n💻 Device Configuration Summary:")
print(f"  Training: {'GPU' if USE_GPU_FOR_TRAINING and torch.cuda.is_available() else 'CPU'}")
print(f"  Evaluation: {'GPU' if USE_GPU_FOR_EVALUATION and torch.cuda.is_available() else 'CPU'}")
print(f"  Export: {'GPU' if USE_GPU_FOR_EXPORT and torch.cuda.is_available() else 'CPU'}")

print(f"\n📋 Configuration:")
print(f"  Source: data/config.py")
print(f"  Config: cfg_mnet")
print(f"  Network: {cfg_mnet['name']}")
print(f"  Architecture: RetinaFace with CBAM")

print(f"\n📜 Scripts:")
print(f"  Training: train.py")
print(f"  Testing: test_widerface.py")
print(f"  Evaluation: widerface_evaluate/evaluation.py")

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"\n📅 {current_time}")
print(f"💻 PyTorch {torch.__version__}")

print(f"\n💡 CONFIGURATION TIPS:")
print(f"  • To use CPU for evaluation: Set USE_GPU_FOR_EVALUATION = False in Cell 2")
print(f"  • To skip training: Set SKIP_TRAINING = True in Cell 2")
print(f"  • To force training: Set FORCE_TRAINING = True in Cell 2")
print(f"  • To change model path: Edit TRAINED_MODEL_PATH in Cell 2")

print(f"\n{'='*60}")
print("✅ NOTEBOOK READY")
print("🔧 Flexible GPU/CPU configuration enabled")
print(f"{'='*60}")